# $$CatBoost\ Tutorial$$

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/python_tutorial.ipynb)

В этом учебнике мы рассмотрим некоторые основные случаи использования catboost, такие как обучение модели, кросс-валидация и предсказание, а также некоторые полезные функции, такие как ранняя остановка, поддержка снимков, важность признаков и настройка параметров.
  
Вы можете запустить этот учебник в среде Google Colaboratory с бесплатным доступом к CPU или GPU. Просто нажмите на эту <a href="https://colab.research.google.com/github/catboost/tutorials/blob/master/python_tutorial.ipynb" target="_blank" title="Colab">ссылку</a>.

## $$Содержание$$
* [1. Подготовка данных](#$$1.\-Data\-Preparation$$)
    * [1.1 Загрузка данных](#1.1-Data-Loading)
    * [1.2 Подготовка признаков](#1.2-Feature-Preparation)
    * [1.3 Разделение данных](#1.3-Data-Splitting)
* [2. Основы CatBoost](#$$2.\-CatBoost\-Basics$$)
    * [2.1 Обучение модели](#2.1-Model-Training)
    * [2.2 Кросс-валидация модели](#2.2-Model-Cross-Validation)
    * [2.3 Применение модели](#2.3-Model-Applying)
* [3. Особенности CatBoost](#$$3.\-CatBoost\-Features$$)
    * [3.1 Использование лучшей модели](#3.1-Using-the-best-model)
    * [3.2 Ранняя остановка](#3.2-Early-Stopping)
    * [3.3 Использование базовой линии](#3.3-Using-Baseline)
    * [3.4 Поддержка снимков](#3.4-Snapshot-Support)
    * [3.5 Пользовательская целевая функция](#3.5-User-Defined-Objective-Function)
    * [3.6 Пользовательская метрика](#3.6-User-Defined-Metric-Function)
    * [3.7 Пошаговое предсказание](#3.7-Staged-Predict)
    * [3.8 Важность признаков](#3.8-Feature-Importances)
    * [3.9 Оценочные метрики](#3.9-Eval-Metrics)
    * [3.10 Сравнение процессов обучения](#3.10-Learning-Processes-Comparison)
    * [3.11 Сохранение модели](#3.11-Model-Saving)
* [4. Настройка параметров](#$$4.\-Parameters\-Tuning$$)

## $$1.\ Подготовка\ данных$$
### 1.1 Установка CatBoost
Если вы еще не установили CatBoost, вы можете сделать это, выполнив команду '!pip install catboost'.  
  
Также вам следует установить пакет ipywidgets и выполнить специальную команду перед запуском jupyter notebook для отображения графиков.

In [ ]:
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

### 1.2 Загрузка данных
Данные для этого учебника можно получить с [этой страницы](https://www.kaggle.com/c/titanic/data) (вам потребуется зарегистрировать аккаунт на Kaggle или просто войти через Facebook или Google+), либо можно использовать catboost.datasets, как показано в коде ниже.

In [ ]:
from catboost.datasets import titanic
import numpy as np

train_df, test_df = titanic()

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 1.3 Подготовка признаков
Прежде всего давайте проверим, сколько пропущенных значений у нас есть:

In [ ]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

Как видим, **`Age`**, **`Cabin`** и **`Embarked`** действительно содержат пропущенные значения, поэтому давайте заполним их числами, сильно отличающимися от их распределений, чтобы модель могла легко их различить и учитывать:

In [ ]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Теперь давайте разделим признаки и целевую переменную:

In [ ]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

Обратите внимание, что наши признаки имеют разные типы — некоторые из них числовые, некоторые категориальные, а некоторые даже просто строки, которые обычно нужно обрабатывать специальным образом (например, кодировать через представление bag-of-words). Но в нашем случае мы можем рассматривать эти строковые признаки просто как категориальные — всю сложную работу выполнит CatBoost. Круто, да? :)

In [ ]:
print(X.dtypes)

categorical_features_indices = np.where(X.dtypes != float)[0]

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


### 1.4 Разделение данных
Давайте разделим обучающие данные на тренировочный и валидационный наборы.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_df

## $$2.\ CatBoost\ Basics$$

Сделаем необходимые импорты.

In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

### 2.1 Обучение модели
Теперь создадим саму модель. Мы будем использовать параметры по умолчанию, так как они почти всегда дают очень хорошую начальную точку. Единственное, что мы хотели бы указать, это параметр `custom_loss`, чтобы иметь возможность видеть, что происходит с точки зрения метрики этой задачи — точности, а также отслеживать logloss, поскольку он будет более плавным для набора данных такого размера.

In [ ]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [ ]:
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
#     logging_level='Verbose',  # you can uncomment this for text output
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Как видите, можно наблюдать за обучением модели через подробный вывод или красивые графики (я определенно выбрал бы второй вариант — просто взгляните на эти графики: можно, например, увеличить интересующие области!)

Мы видим, что наилучшее значение точности **0.8296** (на валидационном наборе) было достигнуто на **150** шаге бустинга.

### 2.2 Кросс-валидация модели

Хорошо валидировать модель, но проводить кросс-валидацию — еще лучше. И тоже с графиками! Без лишних слов:

In [ ]:
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Теперь у нас есть значения наших функций потерь на каждом шаге бустинга, усредненные по 3 фолдам, что должно дать нам более точную оценку производительности модели:

In [ ]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.83±0.02 on step 355


In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8294051627384961


Как видим, наша начальная оценка производительности на одном валидационном фолде была слишком оптимистичной — вот почему кросс-валидация так важна!

### 2.3 Применение модели
Все, что вам нужно сделать, чтобы получить предсказания:

In [ ]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0 0 0 0 1 0 1 0 1 0]
[[0.85473931 0.14526069]
 [0.76313031 0.23686969]
 [0.88972889 0.11027111]
 [0.87876173 0.12123827]
 [0.3611047  0.6388953 ]
 [0.90513381 0.09486619]
 [0.33434185 0.66565815]
 [0.78468564 0.21531436]
 [0.39429048 0.60570952]
 [0.94047549 0.05952451]]


Но давайте попробуем получить лучшие предсказания, и функции CatBoost нам в этом помогут.

## $$3.\ Особенности\ CatBoost$$
Вы могли заметить, что на этапе создания модели я указал не только `custom_loss`, но и параметр `random_seed`. Это было сделано для того, чтобы сделать этот ноутбук воспроизводимым — по умолчанию CatBoost выбирает случайное значение для seed:

In [ ]:
model_without_seed = CatBoostClassifier(iterations=10, logging_level='Silent')
model_without_seed.fit(X, y, cat_features=categorical_features_indices)

print('Random seed assigned for this model: {}'.format(model_without_seed.random_seed_))

Random seed assigned for this model: 0


Давайте определим несколько параметров и создадим `Pool` для удобства. Он хранит всю информацию о наборе данных (признаки, метки, индексы категориальных признаков, веса и многое другое).

In [ ]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': False
}
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

### 3.1 Использование лучшей модели
Если у вас есть валидационный набор, всегда лучше использовать параметр `use_best_model` во время обучения. По умолчанию этот параметр включен. Если он включен, результирующий ансамбль деревьев будет уменьшен до лучшей итерации.

In [ ]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

best_model_params = params.copy()
best_model_params.update({
    'use_best_model': True
})
best_model = CatBoostClassifier(**best_model_params)
best_model.fit(train_pool, eval_set=validate_pool);

print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Best model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, best_model.predict(X_validation))
))

Simple model validation accuracy: 0.7982

Best model validation accuracy: 0.8251


### 3.2 Ранняя остановка
Если у вас есть валидационный набор, то всегда проще и лучше использовать раннюю остановку. Эта функция похожа на предыдущую, но помимо улучшения качества она еще экономит время.

In [ ]:
%%time
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool)

CPU times: user 9.42 s, sys: 368 ms, total: 9.79 s
Wall time: 1.1 s


In [ ]:
%%time
earlystop_params = params.copy()
earlystop_params.update({
    'od_type': 'Iter',
    'od_wait': 40
})
earlystop_model = CatBoostClassifier(**earlystop_params)
earlystop_model.fit(train_pool, eval_set=validate_pool);

CPU times: user 1.52 s, sys: 108 ms, total: 1.62 s
Wall time: 195 ms


In [ ]:
print('Simple model tree count: {}'.format(model.tree_count_))
print('Simple model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, model.predict(X_validation))
))
print('')

print('Early-stopped model tree count: {}'.format(earlystop_model.tree_count_))
print('Early-stopped model validation accuracy: {:.4}'.format(
    accuracy_score(y_validation, earlystop_model.predict(X_validation))
))

Simple model tree count: 500
Simple model validation accuracy: 0.7982

Early-stopped model tree count: 82
Early-stopped model validation accuracy: 0.8072


Таким образом, мы получаем лучшее качество за меньшее время.

Хотя, как было показано ранее, простая схема валидации не точно описывает модель вне обучающей выборки (может быть смещена из-за разделения данных), все же полезно отслеживать динамику улучшения модели — и, как видно из этого примера, действительно хорошо остановить процесс бустинга раньше (до того, как начнется переобучение).

### 3.3 Использование Baseline
Можно использовать результаты предварительного обучения (baseline) для обучения.

In [ ]:
current_params = params.copy()
current_params.update({
    'iterations': 10
})
model = CatBoostClassifier(**current_params).fit(X_train, y_train, categorical_features_indices)
# Get baseline (only with prediction_type='RawFormulaVal')
baseline = model.predict(X_train, prediction_type='RawFormulaVal')
# Fit new model
model.fit(X_train, y_train, categorical_features_indices, baseline=baseline);

### 3.4 Поддержка snapshot'ов
Catboost поддерживает snapshot'ы. Вы можете использовать их для восстановления обучения после прерывания или для начала обучения с предыдущими результатами.

In [ ]:
params_with_snapshot = params.copy()
params_with_snapshot.update({
    'iterations': 5,
    'learning_rate': 0.5,
    'logging_level': 'Verbose'
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)
params_with_snapshot.update({
    'iterations': 10,
    'learning_rate': 0.1,
})
model = CatBoostClassifier(**params_with_snapshot).fit(train_pool, eval_set=validate_pool, save_snapshot=True)

0:	learn: 0.8053892	test: 0.7937220	best: 0.7937220 (0)	total: 1.39ms	remaining: 5.54ms
1:	learn: 0.8008982	test: 0.7982063	best: 0.7982063 (1)	total: 3ms	remaining: 4.5ms
2:	learn: 0.8008982	test: 0.7937220	best: 0.7982063 (1)	total: 4.13ms	remaining: 2.75ms
3:	learn: 0.8113772	test: 0.7892377	best: 0.7982063 (1)	total: 5.52ms	remaining: 1.38ms
4:	learn: 0.8173653	test: 0.8026906	best: 0.8026906 (4)	total: 6.67ms	remaining: 0us

bestTest = 0.802690583
bestIteration = 4

5:	learn: 0.8173653	test: 0.8026906	best: 0.8026906 (4)	total: 8.14ms	remaining: 5.89ms
6:	learn: 0.8248503	test: 0.8026906	best: 0.8026906 (4)	total: 9.64ms	remaining: 4.46ms
7:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 10.8ms	remaining: 2.76ms
8:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 11.4ms	remaining: 1.19ms
9:	learn: 0.8233533	test: 0.8026906	best: 0.8026906 (4)	total: 12.7ms	remaining: 0us

bestTest = 0.802690583
bestIteration = 4



### 3.5 Пользовательская целевая функция
Можно создать собственную целевую функцию. Давайте создадим целевую функцию logloss.

In [ ]:
# for performance reasons it is better to install `numba` package for working with user defined functions
!pip install numba

In [ ]:
class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        # approxes, targets, weights are indexed containers of floats
        # (containers which have only __len__ and __getitem__ defined).
        # weights parameter can be None.
        #
        # To understand what these parameters mean, assume that there is
        # a subset of your dataset that is currently being processed.
        # approxes contains current predictions for this subset,
        # targets contains target values you provided with the dataset.
        #
        # This function should return a list of pairs (der1, der2), where
        # der1 is the first derivative of the loss function with respect
        # to the predicted value, and der2 is the second derivative.
        #
        # In our case, logloss is defined by the following formula:
        # target * log(sigmoid(approx)) + (1 - target) * (1 - sigmoid(approx))
        # where sigmoid(x) = 1 / (1 + e^(-x)).

        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)

        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = (1 - p) if targets[index] > 0.0 else -p
            der2 = -p * (1 - p)

            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [ ]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42,
    loss_function=LoglossObjective(),
    eval_metric=metrics.Logloss()
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

0:	learn: 0.6827074	total: 16.7ms	remaining: 150ms
1:	learn: 0.6723302	total: 30.8ms	remaining: 123ms
2:	learn: 0.6619449	total: 43.3ms	remaining: 101ms
3:	learn: 0.6521466	total: 60.3ms	remaining: 90.5ms
4:	learn: 0.6435227	total: 73.4ms	remaining: 73.4ms
5:	learn: 0.6353848	total: 87.8ms	remaining: 58.5ms
6:	learn: 0.6277210	total: 99.6ms	remaining: 42.7ms
7:	learn: 0.6210282	total: 111ms	remaining: 27.8ms
8:	learn: 0.6141958	total: 123ms	remaining: 13.6ms
9:	learn: 0.6073236	total: 135ms	remaining: 0us


### 3.6 Пользовательская метрика
Также можно создать собственную метрику. Давайте создадим метрику logloss.

In [ ]:
class LoglossMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        # approxes is a list of indexed containers
        # (containers with only __len__ and __getitem__ defined),
        # one container per approx dimension.
        # Each container contains floats.
        # weight is a one dimensional indexed container.
        # target is float.

        # weight parameter can be None.
        # Returns pair (error, weights sum)

        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += -w * (target[i] * approx[i] - np.log(1 + np.exp(approx[i])))

        return error_sum, weight_sum

In [ ]:
model = CatBoostClassifier(
    iterations=10,
    random_seed=42,
    loss_function=metrics.Logloss(),
    eval_metric=LoglossMetric()
)
# Fit model
model.fit(train_pool)
# Only prediction_type='RawFormulaVal' is allowed with custom `loss_function`
preds_raw = model.predict(X_test, prediction_type='RawFormulaVal')

Learning rate set to 0.5
0:	learn: 0.5521578	total: 6.34ms	remaining: 57.1ms
1:	learn: 0.4885686	total: 11.9ms	remaining: 47.5ms
2:	learn: 0.4607664	total: 17.4ms	remaining: 40.5ms
3:	learn: 0.4418819	total: 22.6ms	remaining: 33.9ms
4:	learn: 0.4278162	total: 28.4ms	remaining: 28.4ms
5:	learn: 0.4151036	total: 35.1ms	remaining: 23.4ms
6:	learn: 0.4099336	total: 40.6ms	remaining: 17.4ms
7:	learn: 0.4095363	total: 45.8ms	remaining: 11.5ms
8:	learn: 0.4032867	total: 51.3ms	remaining: 5.7ms
9:	learn: 0.3929586	total: 56.6ms	remaining: 0us


### 3.7 Пошаговое предсказание
Модель CatBoost имеет метод `staged_predict`. Он позволяет вам поэтапно получать предсказания для заданного диапазона деревьев.

In [ ]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
ntree_start, ntree_end, eval_period = 3, 9, 2
predictions_iterator = model.staged_predict(validate_pool, 'Probability', ntree_start, ntree_end, eval_period)
for preds, tree_count in zip(predictions_iterator, range(ntree_start, ntree_end, eval_period)):
    print('First class probabilities using the first {} trees: {}'.format(tree_count, preds[:5, 1]))

First class probabilities using the first 3 trees: [0.53597869 0.41039128 0.42057479 0.64281031 0.46576685]
First class probabilities using the first 5 trees: [0.63722688 0.42492029 0.46209302 0.70926021 0.44280772]
First class probabilities using the first 7 trees: [0.66964764 0.42409144 0.46124982 0.76101033 0.47205986]


### 3.8 Важность признаков
Иногда очень важно понять, какой признак оказал наибольшее влияние на конечный результат. Для этого модель CatBoost имеет метод `get_feature_importance`.

In [ ]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Sex: 59.0040920142686
Pclass: 16.340887169747038
Ticket: 6.028107169932206
Cabin: 3.8347242202560192
Fare: 3.712969667934385
Age: 3.4844512041824824
Parch: 3.378089740355865
Embarked: 2.313999407289956
SibSp: 1.902679406033451
PassengerId: 0.0
Name: 0.0


Это показыват, что признаки **`Sex`** и **`Pclass`** имеют наибольший влияние на результат.

### 3.9 Оценочные метрики
CatBoost имеет метод `eval_metrics`, который позволяет рассчитывать заданные метрики на заданном наборе данных. И, конечно, рисовать их :)

In [ ]:
model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
eval_metrics = model.eval_metrics(validate_pool, [metrics.AUC()], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
print(eval_metrics['AUC'][:6])

[0.8627368774106994, 0.8623176253563642, 0.8602213650846889, 0.8514170719436525, 0.8495723629045783, 0.8569092738554419]


### 3.10 Сравнение процессов обучения
Вы также можете сравнить процессы обучения разных моделей на одном графике.

In [ ]:
model1 = CatBoostClassifier(iterations=100, depth=1, train_dir='model_depth_1/', logging_level='Silent')
model1.fit(train_pool, eval_set=validate_pool)
model2 = CatBoostClassifier(iterations=100, depth=5, train_dir='model_depth_5/', logging_level='Silent')
model2.fit(train_pool, eval_set=validate_pool);

In [ ]:
from catboost import MetricVisualizer
widget = MetricVisualizer(['model_depth_1', 'model_depth_5'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### 3.11 Сохранение модели
Всегда очень удобно иметь возможность сохранить модель на диск (особенно если обучение заняло некоторое время).

In [ ]:
model = CatBoostClassifier(iterations=10, random_seed=42, logging_level='Silent').fit(train_pool)
model.save_model('catboost_model.dump')
model = CatBoostClassifier()
model.load_model('catboost_model.dump');

# $$4.\ Настройка\ параметров$$
Хотя вы всегда можете выбрать оптимальное количество итераций (шагов бустинга) с помощью кросс-валидации и графиков обучения, также важно поиграть с некоторыми параметрами модели, и мы хотели бы уделить особое внимание параметрам `l2_leaf_reg` и `learning_rate`.

В этом разделе мы выберем эти параметры с помощью пакета **`hyperopt`**.

In [ ]:
!pip install hyperopt

In [ ]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=500,
        eval_metric=metrics.Accuracy(),
        random_seed=42,
        verbose=False,
        loss_function=metrics.Logloss(),
    )

    cv_data = cv(
        Pool(X, y, cat_features=categorical_features_indices),
        model.get_params(),
        logging_level='Silent',
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])

    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:59<00:00,  4.79s/trial, best loss: 0.16386083052749711]
{'l2_leaf_reg': 1.0, 'learning_rate': 0.0450866712211308}


Теперь давайте получим все данные кросс-валидации с лучшими параметрами:

In [ ]:
model = CatBoostClassifier(
    l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=500,
    eval_metric=metrics.Accuracy(),
    random_seed=42,
    verbose=False,
    loss_function=metrics.Logloss(),
)
cv_data = cv(Pool(X, y, cat_features=categorical_features_indices), model.get_params())

Training on fold [0/3]

bestTest = 0.8417508418
bestIteration = 262

Training on fold [1/3]

bestTest = 0.8451178451
bestIteration = 269

Training on fold [2/3]

bestTest = 0.8215488215
bestIteration = 284



In [ ]:
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Precise validation accuracy score: 0.8361391694725029


Помните, что с параметрами по умолчанию наш результат кросс-валидации был 0.8283, и таким образом у нас есть (возможно, не статистически значимое) некоторое улучшение.

### Создадим submission
Теперь мы переобучим нашу настроенную модель на всех доступных обучающих данных.

In [ ]:
model.fit(X, y, cat_features=categorical_features_indices)

И, наконец, давайте подготовим файл отправки:

In [ ]:
import pandas as pd
submisstion = pd.DataFrame()
submisstion['PassengerId'] = X_test['PassengerId']
submisstion['Survived'] = model.predict(X_test)

In [ ]:
submisstion.to_csv('submission.csv', index=False)

Наконец, вы можете сделать отправку на [конкурс Titanic Kaggle](https://www.kaggle.com/c/titanic).

Вот и все! Теперь вы можете экспериментировать с CatBoost и выигрывать конкурсы! :)